In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.5. Group Distribution Analysis/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.5. Group Distribution Analysis/Output')
cur_date = "070122"

library(readxl)
library(tidyverse)
library(reshape2)


#reading in file
proteomics_df = data.frame(read_excel("070122_Imputed_Proteomics_Data.xlsx"))
protein_info_df = data.frame(read_excel("List of Proteins Detected_041922.xlsx")) %>%
    rename(Protein = Accession)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(proteomics_df)
head(protein_info_df)

,Set,Treatment,ID,Protein,Value,log2Value
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Set1,CoCulture_Exposed,CoCulture_Exposed1,P02769,498216198704,38.85798
2,Set1,CoCulture_Exposed,CoCulture_Exposed1,P02751,184361941370,37.42375
3,Set1,CoCulture_Exposed,CoCulture_Exposed1,P12763,293003799678,38.09213
4,Set1,CoCulture_Exposed,CoCulture_Exposed1,Q2UVX4,52235883387,35.60432
5,Set1,CoCulture_Exposed,CoCulture_Exposed1,P98160,21618632410,34.33156
6,Set1,CoCulture_Exposed,CoCulture_Exposed1,P12111,18326454885,34.09321


,Protein,Gene.name,Description,GOCC.name,Contaminant
,<chr>,<chr>,<chr>,<chr>,<chr>
1,P02751,FN1,Fibronectin OS=Homo sapiens OX=9606 GN=FN1 PE=1 SV=5,apical plasma membrane;basal lamina;blood microparticle;cell part;cytoplasmic membrane-bounded vesicle lumen;cytoplasmic part;cytoplasmic vesicle part;endoplasmic reticulum-Golgi intermediate compartment;extracellular matrix;extracellular matrix part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region;extracellular region part;extracellular space;extracellular vesicular exosome;fibrinogen complex;intracellular membrane-bounded organelle;intracellular organelle;intracellular organelle part;intracellular part;macromolecular complex;membrane part;membrane-bounded organelle;membrane-bounded vesicle;membrane-enclosed lumen;organelle;organelle lumen;organelle part;plasma membrane part;platelet alpha granule lumen;protein complex;secretory granule lumen;vesicle;vesicle lumen,-
2,P02769,SWISS-PROT:P02769 (Bos taurus) Bovine serum albumin precursor,SWISS-PROT:P02769 (Bos taurus) Bovine serum albumin precursor,NA,+
3,P07996,THBS1,Thrombospondin-1 OS=Homo sapiens OX=9606 GN=THBS1 PE=1 SV=2,cell part;cell surface;cytoplasmic membrane-bounded vesicle;cytoplasmic membrane-bounded vesicle lumen;cytoplasmic part;cytoplasmic vesicle;cytoplasmic vesicle part;endoplasmic reticulum;endoplasmic reticulum lumen;endoplasmic reticulum part;external side of plasma membrane;extracellular matrix;extracellular membrane-bounded organelle;extracellular organelle;extracellular region;extracellular region part;extracellular space;extracellular vesicular exosome;fibrinogen complex;intracellular membrane-bounded organelle;intracellular organelle;intracellular organelle lumen;intracellular organelle part;intracellular part;macromolecular complex;membrane part;membrane-bounded organelle;membrane-bounded vesicle;membrane-enclosed lumen;organelle;organelle lumen;organelle part;plasma membrane part;platelet alpha granule;platelet alpha granule lumen;protein complex;sarcoplasmic reticulum;secretory granule lumen;stored secretory granule;vesicle;vesicle lumen,-
4,Q2UVX4,SWISS-PROT:Q2UVX4 (Bos taurus) Complement C3 precursor,SWISS-PROT:Q2UVX4 (Bos taurus) Complement C3 precursor,NA,+
5,P98160,HSPG2,Basement membrane-specific heparan sulfate proteoglycan core protein OS=Homo sapiens OX=9606 GN=HSPG2 PE=1 SV=4,adherens junction;anchoring junction;basal lamina;cell junction;cell part;cell-substrate adherens junction;cell-substrate junction;cytoplasmic part;extracellular matrix;extracellular matrix part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region;extracellular region part;extracellular space;extracellular vesicular exosome;focal adhesion;Golgi apparatus part;Golgi lumen;intracellular organelle lumen;intracellular organelle part;intracellular part;lysosomal lumen;membrane;membrane-bounded organelle;membrane-bounded vesicle;membrane-enclosed lumen;organelle;organelle lumen;organelle part;plasma membrane;vacuolar lumen;vacuolar part;vesicle,-
6,P12763,SWISS-PROT:P12763 (Bos taurus) Alpha-2-HS-glycoprotein precursor,SWISS-PROT:P12763 (Bos taurus) Alpha-2-HS-glycoprotein precursor,NA,+


In [3]:
proteomics_df = proteomics_df %>%
    # just want a column that specifies the treatment and exposure separately
    separate(Treatment, c("Treatment", "Exposure"), sep = "_") 

head(proteomics_df)

,Set,Treatment,Exposure,ID,Protein,Value,log2Value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Set1,CoCulture,Exposed,CoCulture_Exposed1,P02769,498216198704,38.85798
2,Set1,CoCulture,Exposed,CoCulture_Exposed1,P02751,184361941370,37.42375
3,Set1,CoCulture,Exposed,CoCulture_Exposed1,P12763,293003799678,38.09213
4,Set1,CoCulture,Exposed,CoCulture_Exposed1,Q2UVX4,52235883387,35.60432
5,Set1,CoCulture,Exposed,CoCulture_Exposed1,P98160,21618632410,34.33156
6,Set1,CoCulture,Exposed,CoCulture_Exposed1,P12111,18326454885,34.09321


In [4]:
# splitting by Set
proteomics_df_split = proteomics_df %>%
    group_by(Set) %>%
    group_split()

set1_proteomics_df = proteomics_df_split[[1]]
set2_proteomics_df = proteomics_df_split[[2]]

head(set1_proteomics_df)

Set,Treatment,Exposure,ID,Protein,Value,log2Value
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
Set1,CoCulture,Exposed,CoCulture_Exposed1,P02769,498216198704,38.85798
Set1,CoCulture,Exposed,CoCulture_Exposed1,P02751,184361941370,37.42375
Set1,CoCulture,Exposed,CoCulture_Exposed1,P12763,293003799678,38.09213
Set1,CoCulture,Exposed,CoCulture_Exposed1,Q2UVX4,52235883387,35.60432
Set1,CoCulture,Exposed,CoCulture_Exposed1,P98160,21618632410,34.33156
Set1,CoCulture,Exposed,CoCulture_Exposed1,P12111,18326454885,34.09321


Running t tests to determine if there are statistically significant distribution differences in co-culture unexposed vs. exposed and tri-culture unexposed vs. exposed. 

In [5]:
t_test_values = function(df, set_id){
    # """
    # Running t tests after filtering for set, treatment, exposure, and protein using a loop. 
    # Ultimately using this test to compare proteins (unexposed vs. exposed).

    # :param: subsetted dataframe, empty dataframe
    # :output: a dataframe containing the set, treatment, exposure, protein, u stat, p value, p adj

    # """
    treatments = unique(df$Treatment)
    proteins = unique(df$Protein)
    
    values_df = data.frame()
    # iterating through each tx, protein

    for(i in 1:length(treatments)){
        for(j in 1:length(proteins)){
            # unexposed df
            unexposed_df = df %>%
                filter(Treatment == treatments[i], Exposure == "Vehicle", Protein == proteins[j])
            # exposed df
            exposed_df = df %>%
                filter(Treatment == treatments[i], Exposure == "Exposed", Protein == proteins[j])

            # t test
            t_test = t.test(unexposed_df$log2Value, exposed_df$log2Value)
            
            # calculating FC to get directionality
            FC = log2(mean(exposed_df$Value)/mean(unexposed_df$Value))

            # contains smoking status compared, compartment, cytokine, u stat, and p value
            values_vector = cbind(set_id, treatments[i], proteins[j], FC, t_test$statistic, t_test$p.value)
            values_df = rbind(values_df, values_vector)
            }
        }

    
    # adding col names
    colnames(values_df) = c("Set", "Treatment", "Protein", "log2FC", "Statistic", "P Value")
    
    
   # calculating padj values
    padj_vector = c()
    # adjusting within each treatment
    for (i in 1:length(treatments)){
        `P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")
        padj_vector = c(padj_vector, `P Adj`)
    }
    
    values_df = cbind(values_df, padj_vector) %>%
        rename(`P Adj` = padj_vector)

    values_df$log2FC = as.numeric(values_df$log2FC) 
    values_df$Statistic = as.numeric(values_df$Statistic)
    values_df$`P Value` = as.numeric(values_df$`P Value`) 
    
    return(values_df)
}

In [6]:
# calling fn
set1_t_test = t_test_values(set1_proteomics_df, "Set1")
set2_t_test = t_test_values(set2_proteomics_df, "Set2")

In [7]:
# creating 1 df
t_test_df = rbind(set1_t_test, set2_t_test)

# adding in gene names
final_t_test_df = left_join(t_test_df, protein_info_df %>%
                                select(c("Protein", "Gene.name")))
final_t_test_df = final_t_test_df[, c(1,2,3,8,4,5,6,7)]
head(final_t_test_df)

Joining, by = "Protein"


,Set,Treatment,Protein,Gene.name,log2FC,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,Set1,CoCulture,P02769,SWISS-PROT:P02769 (Bos taurus) Bovine serum albumin precursor,0.17688542,-1.5797374,0.1452796,0.6500055
2,Set1,CoCulture,P02751,FN1,-0.09664782,0.3712160,0.7217335,0.9026101
3,Set1,CoCulture,P12763,SWISS-PROT:P12763 (Bos taurus) Alpha-2-HS-glycoprotein precursor,0.04137128,-0.8905752,0.3980407,0.8425572
4,Set1,CoCulture,Q2UVX4,SWISS-PROT:Q2UVX4 (Bos taurus) Complement C3 precursor,-0.01086488,0.2071797,0.8409804,0.9511124
5,Set1,CoCulture,P98160,HSPG2,-0.37199408,1.3220928,0.2155840,0.7205621
6,Set1,CoCulture,P12111,COL6A3,0.25760674,-1.5293073,0.1735218,0.6683300


In [8]:
# exporting t test results for volcano plot
write.csv(final_t_test_df, paste0(Output,"/", cur_date, "_Statistical_Results_Only.csv"), row.names = FALSE)

In [9]:
# calculating overall avg. abundance and within smoking groups
avg_abundance_df = proteomics_df %>%
    group_by(Protein) %>%
    summarize(Avg_Abundance = mean(log2Value))

# stratified by treatment and exposure
avg_abundance_df_stratified = proteomics_df %>%
    group_by(Treatment, Exposure, Protein) %>%
    mutate(Avg_Abundance_stratified = mean(log2Value)) %>%
    select(-c("Set", "ID", "Value"))

head(avg_abundance_df)
head(avg_abundance_df_stratified)

Protein,Avg_Abundance
<chr>,<dbl>
A0AVT1,23.20928
A1L4H1,25.83883
A2I7N0,25.88141
A8K2U0,18.14997
ENSEMBL.ENSBTAP00000006074,29.08530
ENSEMBL.ENSBTAP00000007350,32.59103


Treatment,Exposure,Protein,log2Value,Avg_Abundance_stratified
<chr>,<chr>,<chr>,<dbl>,<dbl>
CoCulture,Exposed,P02769,38.85798,38.97749
CoCulture,Exposed,P02751,37.42375,37.70872
CoCulture,Exposed,P12763,38.09213,38.00144
CoCulture,Exposed,Q2UVX4,35.60432,35.38864
CoCulture,Exposed,P98160,34.33156,34.09359
CoCulture,Exposed,P12111,34.09321,34.35725


In [10]:
# changing the stratified dfs to making combining easier
# first splitting dfs
split_avg_abundance_stratified = avg_abundance_df_stratified %>%
    group_by(Treatment, Exposure) %>%
    group_split()

# adding treatment to the title
coculture_exposed_avg_abundance = split_avg_abundance_stratified[[1]] %>%
    select(-c("Treatment", "Exposure", "log2Value")) %>%
    rename(Coculture_Exposed.Avg_Abundance = Avg_Abundance_stratified)
coculture_vehicle_avg_abundance = split_avg_abundance_stratified[[2]] %>%
    select(-c("Treatment", "Exposure", "log2Value")) %>%
    rename(Coculture_Vehicle.Avg_Abundance = Avg_Abundance_stratified)
triculture_exposed_avg_abundance = split_avg_abundance_stratified[[3]] %>%
    select(-c("Treatment", "Exposure", "log2Value")) %>%
    rename(Triculture_Exposed.Avg_Abundance = Avg_Abundance_stratified)
triculture_vehicle_avg_abundance = split_avg_abundance_stratified[[4]] %>%
    select(-c("Treatment", "Exposure", "log2Value")) %>%
    rename(Triculture_Vehicle.Avg_Abundance = Avg_Abundance_stratified)

head(coculture_exposed_avg_abundance)

Protein,Coculture_Exposed.Avg_Abundance
<chr>,<dbl>
P02769,38.97749
P02751,37.70872
P12763,38.00144
Q2UVX4,35.38864
P98160,34.09359
P12111,34.35725


In [11]:
# combining into 1 abundance df
stratified_abundance_df = full_join(inner_join(coculture_exposed_avg_abundance, coculture_vehicle_avg_abundance) %>% 
                                    unique(), 
      inner_join(triculture_exposed_avg_abundance, triculture_vehicle_avg_abundance) %>% 
                                    unique())
abundance_df = full_join(avg_abundance_df, stratified_abundance_df)

head(abundance_df)

Joining, by = "Protein"
Joining, by = "Protein"
Joining, by = "Protein"
Joining, by = "Protein"


Protein,Avg_Abundance,Coculture_Exposed.Avg_Abundance,Coculture_Vehicle.Avg_Abundance,Triculture_Exposed.Avg_Abundance,Triculture_Vehicle.Avg_Abundance
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A0AVT1,23.20928,NA,NA,22.84180,23.57677
A1L4H1,25.83883,23.68590,24.25528,27.68968,27.72448
A2I7N0,25.88141,26.04411,25.71871,NA,NA
A8K2U0,18.14997,NA,NA,19.03505,17.26489
ENSEMBL.ENSBTAP00000006074,29.08530,29.45674,29.47218,28.80154,28.61074
ENSEMBL.ENSBTAP00000007350,32.59103,32.71867,32.63214,32.63952,32.37379


Some cells have NA values, because not all proteins were the same between coculture and triculture treatment groups. 

In [12]:
# getting the final statistical df to combine t test and abundance values
# will have one df for each set
final_t_test_df_split = final_t_test_df %>%
    select(-Treatment) %>%
    group_by(Set) %>%
    group_split()


set1_t_test_abundance_df = inner_join(final_t_test_df_split[[1]], abundance_df[,1:4]) %>%
    select(-Set)
set2_t_test_abundance_df = inner_join(final_t_test_df_split[[2]], abundance_df[,c(1,2,5,6)]) %>%
    select(-Set)
head(set1_t_test_abundance_df)

Joining, by = "Protein"
Joining, by = "Protein"


Protein,Gene.name,log2FC,Statistic,P Value,P Adj,Avg_Abundance,Coculture_Exposed.Avg_Abundance,Coculture_Vehicle.Avg_Abundance
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
P02769,SWISS-PROT:P02769 (Bos taurus) Bovine serum albumin precursor,0.17688542,-1.5797374,0.1452796,0.6500055,38.47683,38.97749,38.79991
P02751,FN1,-0.09664782,0.3712160,0.7217335,0.9026101,37.72048,37.70872,37.77246
P12763,SWISS-PROT:P12763 (Bos taurus) Alpha-2-HS-glycoprotein precursor,0.04137128,-0.8905752,0.3980407,0.8425572,37.52956,38.00144,37.95837
Q2UVX4,SWISS-PROT:Q2UVX4 (Bos taurus) Complement C3 precursor,-0.01086488,0.2071797,0.8409804,0.9511124,35.40277,35.38864,35.42594
P98160,HSPG2,-0.37199408,1.3220928,0.2155840,0.7205621,34.16685,34.09359,34.46918
P12111,COL6A3,0.25760674,-1.5293073,0.1735218,0.6683300,34.12207,34.35725,34.05447


In [13]:
# exporting t test results and avg abundances for manuscript
write.csv(set1_t_test_abundance_df, paste0(Output,"/", cur_date, "_Set1_t_test_abundance_data.csv"), row.names = FALSE)
write.csv(set2_t_test_abundance_df, paste0(Output,"/", cur_date, "_Set2_t_test_abundance_data.csv"), row.names = FALSE)

In [14]:
# now creating a df with individual protein abundances
split_proteomics_df = proteomics_df %>%
    select(-c("Treatment", "Exposure", "Value")) %>%
    group_by(Set) %>%
    group_split()

set1_protein_abundance = split_proteomics_df[[1]] %>%
    select(-Set) %>%
    pivot_wider(names_from = ID, values_from = log2Value) 

set2_protein_abundance = split_proteomics_df[[2]] %>%
     select(-Set) %>%
    pivot_wider(names_from = ID, values_from = log2Value) 

head(set1_protein_abundance)

Protein,CoCulture_Exposed1,CoCulture_Exposed2,CoCulture_Exposed3,CoCulture_Exposed4,CoCulture_Exposed5,CoCulture_Exposed6,CoCulture_VehicleCT1,CoCulture_VehicleCT2,CoCulture_VehicleCT3,CoCulture_VehicleCT4,CoCulture_VehicleCT5,CoCulture_VehicleCT6
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
P02769,38.85798,39.14599,39.16376,38.90002,38.68933,39.10784,39.00936,38.67214,38.75845,38.49615,38.87343,38.98993
P02751,37.42375,37.75165,37.85741,37.88589,37.68203,37.65156,37.52669,37.82217,38.20956,38.06975,37.86170,37.14490
P12763,38.09213,38.02277,37.99139,37.93613,38.03726,37.92895,37.99546,37.76154,37.94652,38.00562,38.00721,38.03387
Q2UVX4,35.60432,35.18843,34.83988,35.92622,35.53571,35.23729,35.79829,35.32697,35.22374,35.45133,35.54565,35.20968
P98160,34.33156,33.67026,33.33868,34.35007,34.68056,34.19040,34.63103,34.05924,33.67706,34.80922,34.73756,34.90098
P12111,34.09321,34.24833,34.37701,34.61420,34.42455,34.38622,33.67353,34.30335,34.38698,34.40145,34.24699,33.31450


In [15]:
# exporting indvidual protein abundances
write.csv(set1_protein_abundance, paste0(Output,"/", cur_date, "_Set1_individual_abundance_data.csv"), row.names = FALSE)
write.csv(set2_protein_abundance, paste0(Output,"/", cur_date, "_Set2_individual_abundance_data.csv"), row.names = FALSE)